<a href="https://colab.research.google.com/github/satender1234/music-classify/blob/fix_knn/knn/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [41]:
!pip install python_speech_features

In [42]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np
from collections import defaultdict

In [43]:
def distance(instance1, instance2, k):
  distance =0;
  mm1 = instance1[0]
  cm1 = instance1[1]
  mm2 = instance2[0]
  cm2 = instance2[1]
  distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
  distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
  distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
  distance-= k
  return distance

In [44]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [45]:
def getResponse(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [46]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [47]:

directory = '/content/gdrive/MyDrive/genres'
f= open("my.dat" ,'wb')
i=0

for root, folders, files in os.walk(directory):
  for folder in folders:
    print('Reading folder: ', folder)
    i+=1
    if i==11 :
        break
    for file in os.listdir(directory+"/"+folder):
        (rate,sig) = wav.read(directory+"/"+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)

f.close()

Reading folder:  country
Reading folder:  hiphop
Reading folder:  metal
Reading folder:  rock
Reading folder:  jazz
Reading folder:  blues
Reading folder:  pop
Reading folder:  disco
Reading folder:  reggae
Reading folder:  classical


In [48]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break	

    for x in range(len(dataset)):
        if random.random() <split :			
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])	

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)
print('dataset loaded')

dataset loaded


In [49]:
predictions=[]
k = 5
for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		result = getResponse(neighbors)
		predictions.append(result)
		print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + repr(accuracy) + '%')

> predicted=8, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=9, actual=1
> predicted=1, actual=1
> predicted=9, actual=1
> predicted=1, actual=1
> predicted=9, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=4, actual=1
> predicted=1, actual=1
> predicted=6, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=4, actual=1
> predicted=4, actual=1
> predicted=6, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=9, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=4, actual=1
> predicted=9, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=3, actual=1
> predicted=1, actual=1
> predicted=2, actual=1
> predicted=1, actual=1
> predicted=9, actual=2
> predicted=3, actual=2
> predicted=2, a

In [56]:
results = defaultdict(int)

i = 1
for root, folders, files in os.walk(directory):
  for folder in folders:
    results[i]=folder
    i+=1

# print(results)

(rate,sig) = wav.read('/content/gdrive/MyDrive/genres/jazz/jazz.00077.wav')
mfcc_feat = mfcc(sig,rate,winlen=0.020,appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix,covariance,0)

prediction = getResponse(getNeighbors(dataset ,feature , 5))
print(prediction)
print(results[prediction])

5
jazz
